In [6]:
import time
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)

class LoggerMixin:
    def log(self, message):
        logging.info(f"{self.__class__.__name__}: {message}")

class TimerMixin:
    def time_execution(self, func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            self.log(f"Execution time of {func.__name__}: {end_time - start_time:.4f} seconds")
            return result
        return wrapper

class DataProcessor(LoggerMixin, TimerMixin):
    def process_data(self, data):
        self.log("Starting data processing")
        # Симуляция обработки данных
        time.sleep(2)
        self.log("Data processing complete")
        return [d * 2 for d in data]

    # Применяем декоратор в конструкторе
    def __init__(self):
        self.process_data = self.time_execution(self.process_data)

processor = DataProcessor()
processed_data = processor.process_data([1, 2, 3, 4, 5])
print(processed_data)


INFO:root:DataProcessor: Starting data processing
INFO:root:DataProcessor: Data processing complete
INFO:root:DataProcessor: Execution time of process_data: 2.0015 seconds


[2, 4, 6, 8, 10]


Конечно! Ошибка возникала потому, что `time_execution` использовался как статический метод, а не как метод экземпляра. Вот что я сделал для исправления:

1. **Метод экземпляра:** 
   - Декоратор `time_execution` должен быть методом экземпляра, чтобы иметь доступ к `self` и использовать методы класса, такие как `log`.

2. **Инициализация в конструкторе:**
   - Вместо использования декоратора непосредственно над методом `process_data`, я применил его в конструкторе класса `DataProcessor`. Это позволяет обернуть метод `process_data` в декоратор после создания экземпляра класса.

Таким образом, мы обеспечиваем, что `time_execution` может использовать методы и атрибуты экземпляра, такие как `log`, для выполнения своей функции.

In [ ]:
class TimerMixin:
    def time_execution(self, func):
        def wrapper(*args, **kwargs):
            try:
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                self.log(f"Execution time of {func.__name__}: {end_time - start_time:.4f} seconds")
                return result
            except Exception as e:
                self.log(f"Error in {func.__name__}: {e}")
                raise
        return wrapper

In [ ]:
import time
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LoggerMixin:
    def log_info(self, message):
        logger.info(f"{self.__class__.__name__}: {message}")

    def log_error(self, message):
        logger.error(f"{self.__class__.__name__}: {message}")

class TimerMixin:
    def time_execution(self, func):
        def wrapper(*args, **kwargs):
            try:
                start_time = time.time()
                result = func(*args, **kwargs)
                end_time = time.time()
                self.log_info(f"Execution time of {func.__name__}: {end_time - start_time:.4f} seconds")
                return result
            except Exception as e:
                self.log_error(f"Error in {func.__name__}: {e}")
                raise
        return wrapper

class DataProcessor(LoggerMixin, TimerMixin):
    def __init__(self):
        self.process_data = self.time_execution(self.process_data)

    def process_data(self, data):
        self.log_info("Starting data processing")
        # Симуляция обработки данных
        time.sleep(2)
        if not data:
            raise ValueError("Data cannot be empty")
        self.log_info("Data processing complete")
        return [d * 2 for d in data]

processor = DataProcessor()
try:
    processed_data = processor.process_data([1, 2, 3, 4, 5])
    print(processed_data)
except Exception as e:
    print(f"An error occurred: {e}")
